In [1]:
import json
from dataclasses import dataclass
from typing import Any, AsyncGenerator, Dict, Generator, List, Optional, Union

import pydantic
from anthropic import Anthropic
from anthropic.types import Message as AnthropicMessage
from anthropic.types import MessageParam as AnthropicMessageParam
from openai import NOT_GIVEN, OpenAI
from openai.types.chat import (
    ChatCompletion,
    ChatCompletionChunk,
    ChatCompletionMessage,
    ChatCompletionMessageParam,
    ChatCompletionStreamOptionsParam,
    ChatCompletionSystemMessageParam,
    ChatCompletionToolMessageParam,
    ChatCompletionToolParam,
    ChatCompletionUserMessageParam,
)
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor
from opentelemetry.trace import Status, StatusCode, get_current_span
from opentelemetry.util.types import Attributes
from typing_extensions import assert_never

import openinference.instrumentation as oi
from openinference.instrumentation import (
    TracerProvider,
    get_input_attributes,
    get_llm_attributes,
    get_output_attributes,
    get_span_kind_attributes,
    get_tool_attributes,
    suppress_tracing,
    using_attributes,
)
from openinference.semconv.resource import ResourceAttributes

Either instrument with `TracerProvider` from `openinference.instrumentation`.

In [2]:
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={ResourceAttributes.PROJECT_NAME: "openinference-tracer"})
tracer_provider = TracerProvider(resource=resource)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))
tracer = tracer_provider.get_tracer(__name__)

Or using `phoenix.otel.register` (in which case, comment out cell above and uncomment this cell).

In [ ]:
# from phoenix.otel import register

# tracer_provider = register(protocol="http/protobuf")
# tracer = tracer_provider.get_tracer(__name__)

## LLMs

In [3]:
anthropic_client = Anthropic()
openai_client = OpenAI()

### Context Manager

In [ ]:
with tracer.start_as_current_span("llm-span", openinference_span_kind="llm") as span:
    invocation_parameters = {
        "temperature": 0.5,
    }
    input_messages: List[ChatCompletionMessageParam] = [
        ChatCompletionSystemMessageParam(
            role="system",
            content="You are a helpful assistant.",
        ),
        ChatCompletionUserMessageParam(
            role="user",
            content="Hello, world!",
        ),
    ]
    span.set_llm(
        provider="openai",
        system="openai",
        input_messages=input_messages,
        model_name="gpt-4o",
        invocation_parameters=invocation_parameters,
    )
    span.set_input(input_messages[-1]["content"])
    message = openai_client.chat.completions.create(  # type: ignore[call-overload]
        messages=input_messages,
        model="gpt-4o",
        **invocation_parameters,
    )
    output_message = message.choices[0].message
    assert (token_usage := message.usage) is not None
    assert isinstance(completion_tokens := token_usage.completion_tokens, int)
    assert isinstance(prompt_tokens := token_usage.prompt_tokens, int)
    span.set_status(Status(StatusCode.OK))
    span.set_llm(
        output_messages=[
            {
                "role": output_message.role,
                "content": output_message.content,
            }
        ],
        token_count={
            "completion": completion_tokens,
            "prompt": prompt_tokens,
        },
    )
    span.set_output(output_message.content)

In [ ]:
with tracer.start_as_current_span("llm-span-tool-calls", openinference_span_kind="llm") as span:
    input_messages = [
        ChatCompletionUserMessageParam(
            role="user",
            content="What's the weather like in San Francisco?",
        )
    ]
    invocation_parameters = {
        "temperature": 0.5,
    }
    tools: List[ChatCompletionToolParam] = [
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "finds the weather for a given city",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The city to find the weather for, e.g. 'London'",
                        }
                    },
                    "required": ["city"],
                },
            },
        },
    ]
    span.set_llm(
        provider="openai",
        system="openai",
        input_messages=input_messages,
        model_name="gpt-4o",
        invocation_parameters=invocation_parameters,
    )
    span.set_input(input_messages[-1]["content"])
    message = openai_client.chat.completions.create(  # type: ignore[call-overload]
        model="gpt-4",
        tools=tools,
        messages=input_messages,
        **invocation_parameters,
    )
    span.set_status(Status(StatusCode.OK))
    output_message = message.choices[0].message
    assert (token_usage := message.usage) is not None
    assert isinstance(completion_tokens := token_usage.completion_tokens, int)
    assert isinstance(prompt_tokens := token_usage.prompt_tokens, int)
    assert (tool_calls := output_message.tool_calls)
    assert len(tool_calls) == 1
    tool_call = tool_calls[0]
    span.set_llm(
        output_messages=[
            {
                "role": output_message.role,
                "content": output_message.content,
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                ],
            }
        ],
        token_count={
            "completion": completion_tokens,
            "prompt": prompt_tokens,
        },
    )
    span.set_output(f"{tool_call.function.name}({tool_call.function.arguments})")

with tracer.start_as_current_span("llm-span-content-blocks", openinference_span_kind="llm") as span:
    input_messages.append(output_message.model_dump())
    input_messages.append(
        ChatCompletionToolMessageParam(content="sunny", role="tool", tool_call_id=tool_call.id)
    )
    span.set_llm(
        provider="openai",
        system="openai",
        input_messages=input_messages,
        model_name="gpt-4o",
        invocation_parameters=invocation_parameters,
    )
    span.set_input(input_messages[-1]["content"])
    message = openai_client.chat.completions.create(  # type: ignore[call-overload]
        model="gpt-4",
        tools=tools,
        messages=input_messages,
        **invocation_parameters,
    )
    span.set_status(Status(StatusCode.OK))
    output_message = message.choices[0].message
    assert (token_usage := message.usage) is not None
    assert isinstance(completion_tokens := token_usage.completion_tokens, int)
    assert isinstance(prompt_tokens := token_usage.prompt_tokens, int)
    span.set_status(Status(StatusCode.OK))
    span.set_llm(
        output_messages=[
            {
                "role": output_message.role,
                "content": output_message.content,
            }
        ],
        token_count={
            "completion": completion_tokens,
            "prompt": prompt_tokens,
        },
    )
    span.set_output(output_message.content)

In [ ]:
with tracer.start_as_current_span("llm-span-content-blocks", openinference_span_kind="llm") as span:
    invocation_parameters = {
        "max_tokens": 100,
    }
    span.set_llm(
        provider="anthropic",
        system="anthropic",
        input_messages=[
            {
                "role": "user",
                "contents": [
                    {
                        "type": "image",
                        "image": {
                            "url": "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg",
                        },
                    },
                    {
                        "type": "text",
                        "text": "What is in the above image?",
                    },
                ],
            }
        ],
        model_name="gpt-4o",
        invocation_parameters=invocation_parameters,
    )
    span.set_input("What is in the above image?")
    message = anthropic_client.messages.create(  # type: ignore[call-overload]
        model="claude-3-5-sonnet-20240620",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "url",
                            "url": "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg",
                        },
                    },
                    {"type": "text", "text": "What is in the above image?"},
                ],
            }
        ],
        **invocation_parameters,
    )
    span.set_status(Status(StatusCode.OK))
    token_usage = message.usage
    prompt_tokens = token_usage.input_tokens
    completion_tokens = token_usage.output_tokens
    output_message = message.content[0]
    assert output_message.type == "text"
    output_message_text = output_message.text
    span.set_llm(
        output_messages=[
            {
                "role": message.role,
                "content": output_message_text,
            }
        ],
        token_count={
            "completion": completion_tokens,
            "prompt": prompt_tokens,
        },
    )
    span.set_output(output_message_text)

### Decorator

In [4]:
def get_oi_attributes_from_openai_inputs(
    input_messages: List[ChatCompletionMessageParam],
    invocation_parameters: Dict[str, Any],
    tools: Optional[List[ChatCompletionToolParam]] = None,
) -> Dict[str, Any]:
    oi_input_messages = [
        convert_openai_message_param_to_oi_message(message) for message in input_messages
    ]
    oi_tools = [convert_openai_tool_param_to_oi_tool(tool) for tool in tools or []]
    return {
        **get_input_attributes(
            {
                "input_messages": input_messages,
                "invocation_parameters": invocation_parameters,
            }
        ),
        **get_llm_attributes(
            provider="openai",
            system="openai",
            model_name="gpt-4o",
            input_messages=oi_input_messages,
            invocation_parameters=invocation_parameters,
            tools=oi_tools,
        ),
    }


def convert_openai_message_param_to_oi_message(
    message_param: Union[ChatCompletionMessageParam, ChatCompletionMessage],
) -> oi.Message:
    if isinstance(message_param, ChatCompletionMessage):
        role: str = message_param.role
        oi_message = oi.Message(
            role=role,
        )
        if isinstance(content := message_param.content, str):
            oi_message["content"] = content
        if message_param.tool_calls is not None:
            oi_tool_calls: List[oi.ToolCall] = []
            for tool_call in message_param.tool_calls:
                function = tool_call.function
                oi_tool_calls.append(
                    oi.ToolCall(
                        id=tool_call.id,
                        function=oi.ToolCallFunction(
                            name=function.name,
                            arguments=function.arguments,
                        ),
                    )
                )
            oi_message["tool_calls"] = oi_tool_calls
        return oi_message
    if isinstance(message_param, dict):
        role = message_param["role"]
        assert isinstance(message_param["content"], str)
        content = message_param["content"]
    else:
        assert_never(message_param)
    return oi.Message(role=role, content=content)


def convert_openai_tool_param_to_oi_tool(
    tool_param: ChatCompletionToolParam,
) -> oi.Tool:
    assert tool_param["type"] == "function"
    return oi.Tool(
        json_schema=dict(tool_param),
    )


def get_oi_attributes_from_openai_outputs(
    response: ChatCompletion,
) -> Dict[str, Any]:
    choices = response.choices
    assert len(choices) == 1
    message = response.choices[0].message
    role = message.role
    oi_message = oi.Message(role=role)
    if isinstance(message.content, str):
        oi_message["content"] = message.content
    if isinstance(message.tool_calls, list):
        oi_tool_calls: List[oi.ToolCall] = []
        for tool_call in message.tool_calls:
            tool_call_id = tool_call.id
            function_name = tool_call.function.name
            function_arguments = tool_call.function.arguments
            oi_tool_calls.append(
                oi.ToolCall(
                    id=tool_call_id,
                    function=oi.ToolCallFunction(
                        name=function_name,
                        arguments=function_arguments,
                    ),
                )
            )
        oi_message["tool_calls"] = oi_tool_calls
    output_messages = [oi_message]
    token_usage = response.usage
    assert token_usage is not None
    prompt_tokens = token_usage.prompt_tokens
    completion_tokens = token_usage.completion_tokens
    return {
        **get_llm_attributes(
            output_messages=output_messages,
            token_count={
                "completion": completion_tokens,
                "prompt": prompt_tokens,
            },
        ),
        **get_output_attributes(response),
    }


@tracer.llm(
    process_input=get_oi_attributes_from_openai_inputs,
    process_output=get_oi_attributes_from_openai_outputs,
)
def invoke_llm(
    input_messages: List[ChatCompletionMessageParam],
    invocation_parameters: Dict[str, Any],
    tools: Optional[List[ChatCompletionToolParam]] = None,
) -> ChatCompletion:
    response = openai_client.chat.completions.create(
        model="gpt-4",
        tools=tools or NOT_GIVEN,
        messages=input_messages,
        **invocation_parameters,
    )
    assert isinstance(response, ChatCompletion)
    return response


@tracer.tool
def get_weather(city: str) -> str:
    """
    Imagine you were making an API call here.
    """
    return "sunny"


input_messages = [
    ChatCompletionUserMessageParam(
        role="user",
        content="What's the weather like in San Francisco?",
    )
]
invocation_parameters = {
    "temperature": 0.5,
}
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "finds the weather for a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to find the weather for, e.g. 'London'",
                    }
                },
                "required": ["city"],
            },
        },
    },
]
response = invoke_llm(input_messages, invocation_parameters, tools=tools)
output_message = response.choices[0].message
tool_call = output_message.tool_calls[0]
city = json.loads(tool_call.function.arguments)["city"]
weather = get_weather(city)
print(f"{city} is {weather}")
input_messages.append(output_message)
input_messages.append(
    ChatCompletionToolMessageParam(
        content=weather,
        role="tool",
        tool_call_id=tool_call.id,
    )
)
invoke_llm(input_messages, invocation_parameters)

{
    "name": "invoke_llm",
    "context": {
        "trace_id": "0xc7bb8d54d387366acd95bed9cc786384",
        "span_id": "0x342a2d77030a7360",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-04-14T19:46:55.646785Z",
    "end_time": "2025-04-14T19:46:57.732399Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "input.value": "{\"input_messages\": [{\"role\": \"user\", \"content\": \"What's the weather like in San Francisco?\"}], \"invocation_parameters\": {\"temperature\": 0.5}}",
        "input.mime_type": "application/json",
        "llm.provider": "openai",
        "llm.system": "openai",
        "llm.model_name": "gpt-4o",
        "llm.invocation_parameters": "{\"temperature\": 0.5}",
        "llm.input_messages.0.message.role": "user",
        "llm.input_messages.0.message.content": "What's the weather like in San Francisco?",
        "llm.tools.0.tool.json_schema": "{\"type\": \"func

ChatCompletion(id='chatcmpl-BMK3e21WHYR9QbKDLr6JgKbi3qUIh', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The weather in San Francisco is sunny.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1744660018, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=42, total_tokens=51, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [ ]:
def process_input(
    input_messages: List[AnthropicMessageParam],
    invocation_parameters: Dict[str, Any],
) -> Dict[str, Any]:
    oi_input_messages = [to_oi_message(message) for message in input_messages]
    return {
        **get_input_attributes(
            {
                "input_messages": input_messages,
                "invocation_parameters": invocation_parameters,
            }
        ),
        **get_llm_attributes(
            input_messages=oi_input_messages,
            invocation_parameters=invocation_parameters,
        ),
    }


def to_oi_message(message: AnthropicMessageParam) -> oi.Message:
    role = message["role"]
    content = message["content"]
    if isinstance(content, str):
        return oi.Message(role=role, content=content)

    contents: List[oi.MessageContent] = []
    for content_block in content:
        if not isinstance(content_block, dict):
            raise NotImplementedError("Only typed dict message params are supported")
        if (content_type := content_block["type"]) == "text":
            assert isinstance(text := content_block.get("text"), str)
            contents.append(oi.TextMessageContent(type="text", text=text))
        elif content_type == "image":
            assert isinstance(source := content_block.get("source"), dict)
            assert isinstance(url := source.get("url"), str)
            contents.append(
                oi.ImageMessageContent(
                    type="image",
                    image=oi.Image(url=url),
                )
            )
        else:
            raise NotImplementedError("Only text and image message content blocks are supported")
    return oi.Message(role=role, contents=contents)


def process_output(message: AnthropicMessage) -> Dict[str, Any]:
    assert len(message.content) == 1
    content_block = message.content[0]
    assert content_block.type == "text"
    content_block_text = content_block.text
    return {
        **get_llm_attributes(
            output_messages=[
                {
                    "role": message.role,
                    "content": content_block_text,
                }
            ],
            token_count={
                "completion": message.usage.output_tokens,
                "prompt": message.usage.input_tokens,
            },
        ),
        **get_output_attributes(content_block_text),
    }


@tracer.llm(
    process_input=process_input,
    process_output=process_output,
)
def get_image_description(
    input_messages: List[AnthropicMessageParam], invocation_parameters: Dict[str, Any]
) -> AnthropicMessage:
    output_message = anthropic_client.messages.create(
        model="claude-3-5-sonnet-20240620",
        messages=input_messages,
        **invocation_parameters,
    )
    return output_message


response = get_image_description(
    input_messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "url",
                        "url": "https://upload.wikimedia.org/wikipedia/commons/a/a7/Camponotus_flavomarginatus_ant.jpg",
                    },
                },
                {"type": "text", "text": "What is in the above image?"},
            ],
        }
    ],
    invocation_parameters={"temperature": 0.5, "max_tokens": 100},
)

In [ ]:
def get_attributes_from_generator_outputs(outputs: List[ChatCompletionChunk]) -> Attributes:
    role: Optional[str] = None
    content = ""
    oi_token_count = oi.TokenCount()
    for chunk in outputs:
        if choices := chunk.choices:
            assert len(choices) == 1
            delta = choices[0].delta
            if isinstance(delta.content, str):
                content += delta.content
            if isinstance(delta.role, str):
                role = delta.role
        if (usage := chunk.usage) is not None:
            if (prompt_tokens := usage.prompt_tokens) is not None:
                oi_token_count["prompt"] = prompt_tokens
            if (completion_tokens := usage.completion_tokens) is not None:
                oi_token_count["completion"] = completion_tokens
    oi_messages = []
    if role and content:
        oi_messages.append(oi.Message(role=role, content=content))
    return {
        **get_llm_attributes(
            output_messages=oi_messages,
            token_count=oi_token_count,
        ),
        **get_output_attributes(content),
    }


@tracer.llm(
    process_input=get_oi_attributes_from_openai_inputs,
    process_output=get_attributes_from_generator_outputs,
)
def sync_chat_stream(
    input_messages: List[ChatCompletionMessageParam],
    invocation_parameters: Dict[str, Any],
) -> Generator[str, None, None]:
    stream = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=input_messages,
        stream=True,
        stream_options=ChatCompletionStreamOptionsParam(include_usage=True),
        **invocation_parameters,
    )
    for chunk in stream:
        yield chunk


input_messages = [
    ChatCompletionUserMessageParam(
        role="user",
        content="Who won the World Cup in 2022?",
    )
]
invocation_parameters = {
    "temperature": 0.5,
}
for chunk in sync_chat_stream(input_messages, invocation_parameters):
    print(chunk)

In [ ]:
@tracer.llm(
    process_input=get_oi_attributes_from_openai_inputs,
    process_output=get_attributes_from_generator_outputs,
)
async def async_chat_stream(
    input_messages: List[ChatCompletionMessageParam],
    invocation_parameters: Dict[str, Any],
) -> AsyncGenerator[str, None]:
    stream = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=input_messages,
        stream=True,
        stream_options=ChatCompletionStreamOptionsParam(include_usage=True),
        **invocation_parameters,
    )
    for chunk in stream:
        yield chunk


input_messages = [
    ChatCompletionUserMessageParam(
        role="user",
        content="Who won the World Cup in 2022?",
    )
]
invocation_parameters = {
    "temperature": 0.5,
}
async for chunk in async_chat_stream(input_messages, invocation_parameters):
    print(chunk)

## Chains

In [4]:
with tracer.start_as_current_span(
    "chain-span-with-plain-text-io",
    openinference_span_kind="chain",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_status(Status(StatusCode.OK))

### Context Manager

In [5]:
with tracer.start_as_current_span(
    "chain-span-with-json-io",
    openinference_span_kind="chain",
) as span:
    span.set_input(
        {"input-key": "input-value"},
    )
    span.set_output(
        json.dumps({"output-key": "output-value"}),
        mime_type="application/json",
    )
    span.set_status(Status(StatusCode.OK))

In [6]:
with tracer.start_as_current_span(
    "chain-span-with-attribute-getters",
    attributes={
        **get_span_kind_attributes("chain"),
        **get_input_attributes("input"),
    },
) as span:
    span.set_attributes(get_output_attributes("output"))
    span.set_status(Status(StatusCode.OK))

In [7]:
class InputModel(pydantic.BaseModel):
    input: str


@dataclass
class OutputModel:
    output: str


with tracer.start_as_current_span(
    "chain-span-with-pydantic-input-and-dataclass-output",
    openinference_span_kind="chain",
) as span:
    span.set_input(InputModel(input="input"))
    span.set_output(OutputModel(output="output"))
    span.set_status(Status(StatusCode.OK))

### Decorator

In [ ]:
@tracer.chain
def decorated_chain_with_plain_text_output(input: str) -> str:
    return "output"


decorated_chain_with_plain_text_output("input")

In [ ]:
@tracer.chain
def decorated_chain_with_json_output(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated_chain_with_json_output("input")

In [ ]:
@tracer.chain()
def decorated_chain_with_no_parameters(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated_chain_with_no_parameters("input")

In [ ]:
@tracer.chain(name="decorated-chain-with-overriden-name")
def this_name_should_be_overriden(input: str) -> Dict[str, Any]:
    return {"output": "output"}


this_name_should_be_overriden("input")

In [ ]:
def chain_with_decorator_applied_as_function(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated = tracer.chain(chain_with_decorator_applied_as_function)
decorated("input")

In [ ]:
def this_name_should_be_overriden_with_decorator_applied_as_function_with_parameters(
    input: str,
) -> Dict[str, Any]:
    return {"output": "output"}


decorated = tracer.chain(
    name="decorated-chain-with-decorator-applied-as-function-with-overriden-name"
)(this_name_should_be_overriden_with_decorator_applied_as_function_with_parameters)
decorated("input")

In [ ]:
@tracer.chain
async def decorated_async_chain(input: str) -> str:
    return "output"


await decorated_async_chain("input")  # type: ignore[top-level-await]

In [ ]:
@tracer.chain
def decorated_chain_with_error(input: str) -> str:
    raise ValueError("error")


try:
    decorated_chain_with_error("input")
except ValueError as e:
    print(e)

In [ ]:
@tracer.chain
def decorated_chain_with_child_span(input: str) -> str:
    with tracer.start_as_current_span(
        "child-span",
        openinference_span_kind="chain",
        attributes=get_input_attributes("child-span-input"),
    ) as child_span:
        output = "output"
        child_span.set_output(output)
        child_span.set_status(Status(StatusCode.OK))
        return output


decorated_chain_with_child_span("input")

In [ ]:
@tracer.chain
def decorated_chain_with_child_span_error(input: str) -> str:
    with tracer.start_as_current_span(
        "child-span",
        openinference_span_kind="chain",
        attributes=get_input_attributes("child-span-input"),
    ):
        raise ValueError("error")


try:
    decorated_chain_with_child_span_error("input")
except ValueError as e:
    print(e)

In [ ]:
class ChainRunner:
    @tracer.chain
    def decorated_chain_method(self, input1: str, input2: str) -> str:
        return "output"


chain_runner = ChainRunner()
chain_runner.decorated_chain_method("input1", "input2")

In [ ]:
@tracer.chain
def decorated_chain_with_input_and_output_set_inside_the_wrapped_function(input: str) -> str:
    span = get_current_span()
    span.set_input("overridden-input")  # type: ignore[attr-defined]
    span.set_output("overridden-output")  # type: ignore[attr-defined]
    return "output"


decorated_chain_with_input_and_output_set_inside_the_wrapped_function("input")

### Suppress Tracing

In [20]:
with suppress_tracing():
    with tracer.start_as_current_span(
        "THIS-SPAN-SHOULD-NOT-BE-TRACED",
        openinference_span_kind="chain",
    ) as span:
        span.set_input("input")
        span.set_output("output")
        span.set_status(Status(StatusCode.OK))

In [21]:
@tracer.chain
def decorated_chain_with_suppress_tracing(input: str) -> str:
    return "output"


with suppress_tracing():
    decorated_chain_with_suppress_tracing("input")

### Context Attributes

In [22]:
with using_attributes(session_id="123"):
    with tracer.start_as_current_span(
        "chain-span-with-context-attributes",
        openinference_span_kind="chain",
    ) as span:
        span.set_input("input")
        span.set_output("output")
        span.set_status(Status(StatusCode.OK))

In [23]:
@tracer.chain
def decorated_chain_with_context_attributes(input: str) -> str:
    return "output"


with using_attributes(session_id="123"):
    decorated_chain_with_context_attributes("input")

## Agents

### Context Managers

In [24]:
with tracer.start_as_current_span(
    "agent-span-with-plain-text-io",
    openinference_span_kind="agent",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_status(Status(StatusCode.OK))

### Decorators

In [ ]:
@tracer.agent
def decorated_agent(input: str) -> str:
    return "output"


decorated_agent("input")

## Tools

### Context Managers

In [26]:
with tracer.start_as_current_span(
    "tool-span",
    openinference_span_kind="tool",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_tool(
        name="tool-name",
        description="tool-description",
        parameters={"input": "input"},
    )
    span.set_status(Status(StatusCode.OK))

In [27]:
with tracer.start_as_current_span(
    "tool-span-with-getter",
    openinference_span_kind="tool",
) as span:
    span.set_attributes(
        get_tool_attributes(
            name="tool-name",
            description="tool-description",
            parameters={"input": "input"},
        )
    )
    span.set_status(Status(StatusCode.OK))

In [28]:
@tracer.tool
def decorated_tool(input1: str, input2: int) -> None:
    """
    tool-description
    """


decorated_tool("input1", 1)

In [29]:
@tracer.tool
async def decorated_tool_async(input1: str, input2: int) -> None:
    """
    tool-description
    """


await decorated_tool_async("input1", 1)  # type: ignore[top-level-await]

In [30]:
@tracer.tool(
    name="decorated-tool-with-overriden-name",
    description="overriden-tool-description",
)
def this_tool_name_should_be_overriden(input1: str, input2: int) -> None:
    """
    this tool description should be overriden
    """


this_tool_name_should_be_overriden("input1", 1)

In [ ]:
@tracer.tool
def tool_with_changes_inside_the_wrapped_function(input1: str, input2: int) -> str:
    span = get_current_span()
    print(type(span))
    span.set_input("inside-input")  # type: ignore[attr-defined]
    span.set_output("inside-output")  # type: ignore[attr-defined]
    span.set_tool(  # type: ignore[attr-defined]
        name="inside-tool-name",
        description="inside-tool-description",
        parameters={"inside-input": "inside-input"},
    )
    return "output"


tool_with_changes_inside_the_wrapped_function("input1", 1)